In [1]:
import lxml.etree as ET
from transformers import Pipeline

In [2]:
tree = ET.parse(r'/home/tensorboard/Escritorio/Documentos /1. D4R/XML files/27_Reina_Marlorat_Suplicacion_1567_pseudoTEI_toTransformers.xml') # type: ignore
root = tree.getroot()

In [3]:
text = ''
for e in root.iter(): 
    if e.tag == '{http://www.tei-c.org/ns/1.0}div':
        div_container =e
        for regular in div_container.iter():
            if regular.tag == '{http://www.tei-c.org/ns/1.0}reg':
                text+= regular.text+' ' if regular.text is not None else ''


        

In [4]:
text_list = text.split('.')

In [5]:
text_list = [i+'.' for i in text_list if i != ' ']
]

In [6]:
model_checkpoint = '/home/tensorboard/Escritorio/Documentos /1. D4R/8. Models/Full-Linear-NER-Religious-roberta-base-bne-2e-05-lr-10-epochs/checkpoint-3000'
tokenizer_checkpoint = 'PlanTL-GOB-ES/roberta-base-bne'

In [11]:
import torch
from transformers import pipeline
from typing import List, Dict

def ner_inference(
        text: str,
        model_name: str,
        tokenizer_name: str
) -> Dict[str, List[str]]:
    """
    This function performs named entity recognition (NER) on a given text using a pre-trained NER model.

    Args:
        text (str): The input text to perform NER on.
        model_name (str): The name of the pre-trained NER model to use.
        tokenizer_name (str): The name of the tokenizer to use with the model.

    Returns:
        Dict[str, List[str]]: A dictionary containing the input text and the corresponding NER labels.
    """
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    # Initialize the NER pipeline
    classifier = pipeline('ner', model_name, tokenizer=tokenizer_name, device=device)

    # Perform NER on the input text
    entities = classifier(text)

    # If there are any entities detected, process them
    if entities:
        # Initialize an empty list to store the processed entities
    
        
        tokens = []

        # Iterate over each entity and replace any special characters
        for i in entities:
            word = i['word']
            word = word.replace('Ã¡', 'á')
            word = word.replace('Ã©', 'é')
            word = word.replace('ÃŃ', 'í')
            word = word.replace('Ã³', 'ó')
            word = word.replace('Ãº', 'ú')
            word = word.replace('Ã±', 'ñ')
            entity = i['entity']
            tokens.append([word, entity])

        # Initialize an empty list to store the result
        joined_words = []

        # Initialize a variable to store the previous word
        prev_word = ""
        
        # Iterate over each word in the list
        for word, label in tokens:
            try:
            # If the word starts with 'Ġ', add it to the result list
                if word.startswith('Ġ'):
                    joined_words.append([word, label])
                    prev_word = word
                # If the word does not start with 'Ġ', concatenate it with the previous word
                else:
                    # Join the current word with the previous word and add it to the result list
                    joined_words[-1][0] += word
                    # Update the previous word to the newly joined word
                    prev_word = joined_words[-1][0]
            except IndexError:
                print(f'error with word {word}')
                pass

        # Remove the 'Ġ' character from the beginning of each word
        joined_words = [[i[0].replace('Ġ', ''), i[1]] for i in joined_words]

        # Split the input text into individual words
        string_text = text.split()

        # Initialize a list to store the NER labels
        inference_list = ['O'] * len(string_text)

        # Iterate over each word in the input text
        for t in string_text:
            # Iterate over each processed entity
            for li in joined_words:
                # If the current word matches the current entity, update the NER label
                if t == li[0]:
                    inference_list[string_text.index(t)] = li[1]

        # Create a dictionary containing the input text and the corresponding NER labels
        final_dict = {'text': string_text,
                      'entities': inference_list}

        # Return the dictionary
        return final_dict

    # If there are no entities detected, return the input text with 'O' labels
    else:
        string_text = text.split()
        inference_list = ['O'] * len(string_text)
        final_dict = {'text': string_text,
                      'entities': inference_list}
        return final_dict

In [12]:
docs = [ner_inference(text=i, model_name=model_checkpoint, tokenizer_name=tokenizer_checkpoint) for i in text_list]

error with word es
error with word adores
error with word uria
error with word enso
error with word ación
error with word tul
error with word óstoles
error with word tancia
error with word f
error with word tificado
error with word óstoles
error with word ilio
error with word unción
error with word ración
error with word ar
error with word ar
error with word qu
error with word idades
error with word ecil
error with word idad
error with word óstoles
error with word ración
error with word um
error with word nia
error with word fijo
error with word idad
error with word aritos
error with word ación
error with word ción
error with word tu
error with word per
error with word ios
error with word tamiento
error with word is
error with word terías
error with word la
error with word tría
error with word ogados
error with word ios
error with word ige
error with word á
error with word nal


In [13]:
text = [i['text'] for i in docs]

In [14]:
tags = [i['entities'] for i in docs]

In [15]:
import pandas as pd
dict_ = {'text': text, 'entities': tags}
pd.DataFrame.from_dict(dict_)

,text,entities
0,"[Suplicación, e, información, que, fue, presen...","[O, O, O, O, O, O, O, O, PER, PER, PER, O, O, ..."
1,"[Obra, muy, provechosa, para, la, salud, de, l...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
2,"[Compuesta, por, el, maestro, Juan, de, Monte,...","[O, O, O, O, PER, PER, PER, O, O, O, O, GOD, O..."
3,"[Ítem, Un, sermón, de, la, Cruz, .]","[O, O, GOD, O, GOD, GOD, O]"
4,"[1567, .]","[O, O]"
...,...,...
2810,"[Otro, Sermón, falta, aquí, sobre, esta, misma...","[O, GOD, O, O, O, O, O, O, O, O, O, O, O, O, O..."
2811,"[Mas, saldrá, presto, ,, placiendo, a, Dios, .]","[O, O, O, O, O, O, GOD, O]"
2812,"[Entre, tanto, el, lector, goce, y, aprovéches...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2813,"[En, Cosmópoli, por, Cristóforo, Filaleto, .]","[O, GOD, O, PER, PER, O]"


In [16]:
df = pd.DataFrame.from_dict(dict_)

In [18]:
df.to_excel('inference_29-05-24.xlsx')

In [1]:
import pandas as pd
import ast

df = pd.read_excel('/home/tensorboard/Escritorio/Documentos /1. D4R/3. NER/Inference & Testing/inference_29-05-24.xlsx')

In [3]:
df = df.drop('Unnamed: 0', axis=1)

In [5]:
df['entities'] = df['entities'].apply(ast.literal_eval)
df['text'] = df['text'].apply(ast.literal_eval)

In [10]:
x = df['entities'].to_list()
y = df['text'].to_list()

In [30]:
complete = []
symbols_dictionary_full = {
    '∆':'GOD',
    'ƒ':'JUS',
    '©':'CHRI',
    '$':'SACRA',
    '*':'HERESY',
    '%':'PER',
    '#':'PLACE',
    '@':'ORG'

                      }
for tokens, tags in zip(y, x):
    individual = []
    for tok, ta in zip(tokens, tags):
        if ta == 'GOD':
            individual.append(f'∆{tok}∆')

        elif ta == 'JUS':
            individual.append(f'ƒ{tok}ƒ')

        elif ta == 'CHRI':
            individual.append(f'©{tok}©')

        elif ta == 'SACRA':
            individual.append(f'${tok}$')

        elif ta == 'HERESY':
            individual.append(f'*{tok}*')

        elif ta == 'PER':
            individual.append(f'%{tok}%')

        elif ta == 'PLACE':
            individual.append(f'#{tok}#')

        elif ta == 'ORG':
            individual.append(f'@{tok}@')

        else:
            individual.append(tok)
    complete.append(individual)

        

In [31]:
tagged_text = [' '.join(i) for i in complete]

In [32]:
tagged_text = [i.replace('% %', ' ') for i in tagged_text]
tagged_text = [i.replace('© ©', ' ') for i in tagged_text]
tagged_text = [i.replace('ƒ ƒ', ' ') for i in tagged_text]
tagged_text = [i.replace('$ $', ' ') for i in tagged_text]
tagged_text = [i.replace('* *', ' ') for i in tagged_text]
tagged_text = [i.replace('∆ ∆', ' ') for i in tagged_text]
tagged_text = [i.replace('@ @', ' ') for i in tagged_text]
tagged_text = [i.replace('# #', ' ') for i in tagged_text]


['Suplicación e información que fue presentada a la %reina de Francia% por los fieles ©cristianos© de aquel reino : En la cual se trata , Del $sacrificio$ de la ©misa© Del $santísimo sacramento$ del $altar$ Del ƒpurgatorioƒ De la ©invocación© de los ©santos© De la ©veneración© y ©culto© de las ©imágenes© .',
 'Obra muy provechosa para la salud de las ánimas contra los errores de estos tiempos .',
 'Compuesta por el maestro %Juan de Monte% , doctor en sacra ∆Teología∆ , Trasladada de francés en romance castellano , con una exhortación del intérprete al lector .',
 'Ítem Un ∆sermón∆ de ∆la Cruz∆ .',
 '1567 .',
 'El intérprete al lector .',
 'La información y suplicación contenida en este tratado fue presentada en francés el año %de% 1561 a la %reina% de %Francia% , en nombre de los que en aquel reino eran perseguidos por la ©religión© .',
 'En ella muestran la verdad y pureza de la ƒdoctrinaƒ que profesan y las causas que los han forzado a abrazarla .',
 'El motivo que tuvieron para pres

In [35]:
with open('27_Reina_Marlorat_Suplicacion_1567_tagged.txt', 'w', encoding='utf-8') as f:

    f.writelines([f"{item}\n" for item in tagged_text])
    f.close()




In [3]:
list_1 = ['yo', 'soy', 'Héctor', 'López', 'Hidalgo']
list_2 = ['O', 'O', 'PER', 'PER', 'PER']
dict_ ={'PER': '%'}

result_list = []
for item1, item2 in zip(list_1, list_2):
    if item2 == 'PER':
        result_list.append(dict_[item2] + item1)
    else:
        result_list.append(item1)

result_string = ' '.join(result_list[0:2] + ['%'.join(result_list[2:])])
print(result_string)

yo soy %Héctor%%López%%Hidalgo


In [35]:
list_1 = ['yo', 'soy', 'Héctor', 'López', 'Hidalgo']
list_2 = ['O', 'O', 'PER', 'PER', 'PER']
dict_ = {'%': 'PER'}

def generate_string_tagged(list_1, list_2, symbols):
    result = ''

    for word, tag in zip(list_1, list_2):
            if tag == v:
                print(v)
                result += f'{k}{word}{k}'
                break
            else:
                result += f'{word} '
                break
    result = result.replace(r'%%', ' ')
    return result.strip()




In [36]:
string = generate_string_tagged(list_1, list_2, symbols_dictionary_full)
print(string)

yo yo yo yo yo yo yo yo
